In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Portfolio look-through in LUSID

Attributes
----------
entitlements
property values
transactions
"""

toggle_code("Toggle Docstring")

# Property value entitlements

This notebook demonstrates the use of policies to grant access to property values in LUSID. There is an associated Knowledge Base article [here](#).

Table of contents:
1. Setup
2. Prepare data
3. Demonstrate entitlements

---

## 1. Setup

To start, let's import the libraries and initialise the APIs we'll use in the notebook.

In [2]:
# Import Libraries
import os
import random
import math
import json
import pytz

from datetime import datetime, timedelta
from typing import List, Tuple, Dict
from collections import namedtuple

import finbourne_access
import lusid
import pandas as pd

from finbourne_access import models as access_models
from finbourne_access.utilities.api_client_builder import ApiClientBuilder
from lusid import models as models
from lusidjam import RefreshingToken
from IPython.core.display import HTML

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

lusid_api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

api_client = lusid_api_factory.api_client

lusid_api_url = api_client.configuration.host
access_api_url = lusid_api_url[: lusid_api_url.rfind("/") + 1] + "access"

access_api_factory = finbourne_access.utilities.ApiClientFactory(
    token=api_client.configuration.access_token,
    api_url=access_api_url,
    app_name="demo"
)

api_status = pd.DataFrame(
    lusid_api_factory.build(lusid.api.ApplicationMetadataApi)
    .get_lusid_versions()
    .to_dict()
)

display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.6707.0,0.5.2093,"{'relation': 'RequestLogs', 'href': 'http://in..."


In [3]:
scope = "PropertyValueEntitlements"
portfolio_code = "PropertyEntitlementsPortfolioCode"
portfolio_name = "Property entitlements portfolio"

In [4]:
# Initialise all APIs used in the notebook
transaction_portfolios_api = lusid_api_factory.build(lusid.TransactionPortfoliosApi)
portfolios_api = lusid_api_factory.build(lusid.PortfoliosApi)
instruments_api = lusid_api_factory.build(lusid.InstrumentsApi)
properties_api = lusid_api_factory.build(lusid.PropertyDefinitionsApi)

policies_api = access_api_factory.build(finbourne_access.PoliciesApi)
roles_api = access_api_factory.build(finbourne_access.RolesApi)

---

## 2. Prepare data

To demonstrate the entitlements, let's first prepare the data. 

Below, we create a new TransactionPortfolio denominated in GBP, as well as a set of five instruments identified by their FIGIs.

### 2.1 Create portfolio

In [5]:
# Create the portfolio request.
create_portfolio_request = models.CreateTransactionPortfolioRequest(
    display_name=portfolio_name,
    code=portfolio_code,
    base_currency="GBP",
)

try:
    # Make the call to the API.
    transaction_portfolios_api.create_portfolio(scope, create_portfolio_request)
except lusid.ApiException as e:
    detail = json.loads(e.body)
    if detail["code"] != 112: # PortfolioWithIdAlreadyExists
        raise e

### 2.2 Create instruments

In [6]:
InstrumentSpec = namedtuple("InstrumentSpec", ["Figi", "Name"])

instruments = [
    InstrumentSpec("BBG000FD8G46", "HISCOX LTD"),
    InstrumentSpec("BBG000DW76R4", "ITV PLC"),
    InstrumentSpec("BBG000PQKVN8", "MONDI PLC"),
    InstrumentSpec("BBG000BDWPY0", "NEXT PLC"),
    InstrumentSpec("BBG000BF46Y8", "TESCO PLC"),
]

instruments_to_create = {
    i.Figi: models.InstrumentDefinition(
        name=i.Name, identifiers={"Figi": models.InstrumentIdValue(value=i.Figi)}
    )
    for i in instruments
}

response = instruments_api.upsert_instruments(request_body=instruments_to_create)

instrument_ids = sorted([i.lusid_instrument_id for i in response.values.values()])

instrument_ids

['LUID_1Y73CSEM',
 'LUID_75KN4FH2',
 'LUID_B187XORN',
 'LUID_H4ZGVTK5',
 'LUID_X59ZTOIH']

### 2.3 Add transactions to the portfolio

We want to exemplify the property value entitlements on transactions, so let's add some to our portfolio. The amounts and considerations here are randomised, but typically they would be loaded in externally (for example, from a .csv file).

In [7]:
# Prepare transaction requests
transactions = [
    models.TransactionRequest(
        transaction_id=f"TransactionId_{_id}",
        type="Buy",
        instrument_identifiers={"Instrument/default/LusidInstrumentId": _id},
        transaction_date=datetime.now(pytz.UTC).isoformat(),
        settlement_date=(datetime.now(pytz.UTC) + timedelta(days=2)).isoformat(),
        units=math.floor(random.random() * 100),
        total_consideration=lusid.CurrencyAndAmount(
            math.floor(random.random() * 1000), "GBP"
        ),
    )
    for _id in instrument_ids
]

transaction_portfolios_api.upsert_transactions(scope, portfolio_code, transactions)

{'href': 'https://inwaves.lusid.com/api/api/transactionportfolios/PropertyValueEntitlements/PropertyEntitlementsPortfolioCode/transactions?asAt=2021-04-09T08%3A19%3A33.2112230%2B00%3A00',
 'links': [{'description': None,
            'href': 'https://inwaves.lusid.com/api/api/portfolios/PropertyValueEntitlements/PropertyEntitlementsPortfolioCode?effectiveAt=2021-04-09T08%3A19%3A32.8724770%2B00%3A00&asAt=2021-04-09T08%3A19%3A33.2112230%2B00%3A00',
            'method': 'GET',
            'relation': 'Root'},
           {'description': None,
            'href': 'https://inwaves.lusid.com/api/api/schemas/entities/UpsertPortfolioTransactionsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://inwaves.lusid.com/app/insights/logs/0HM7QK7ODIN32:00000016',
            'method': 'GET',
            'relati

### 2.4 Add a property to the transactions

To add a property, we first create a property definition (the `PropertyDefinition` schema in the [API](https://www.lusid.com/api/swagger/index.html)). This gives LUSID information about the property type.

We then add a static value of "entitled" against this new property for all the transactions we inserted above.

In [8]:
# Create a property definition for key Transaction/PropertyValueEntitlements/PropertyValueEntitlement
property_domain = "Transaction"
property_code = "PropertyValueEntitlement"

try:
    properties_api.create_property_definition(
        create_property_definition_request=models.CreatePropertyDefinitionRequest(
            domain=property_domain,
            scope=scope,
            code=property_code,
            display_name=property_code,
            life_time="Perpetual",
            value_required=False,
            data_type_id=models.resource_id.ResourceId(
                scope="system",
                code="string",
            ),
        )
    )
except lusid.ApiException as e:
    detail = json.loads(e.body)
    if detail["code"] != 124:  # "PropertyAlreadyExists"
        raise e

perpetual_property = models.PerpetualProperty(
    f"{property_domain}/{scope}/{property_code}",
    models.PropertyValue(label_value="entitled"),
)

# Add the property to the transactions.
for tran in transactions:
    transaction_portfolios_api.upsert_transaction_properties(
        scope=scope,
        code=portfolio_code,
        transaction_id=tran.transaction_id,
        request_body={f"{property_domain}/{scope}/{property_code}": perpetual_property},
    )

---

## 3. Demonstrate entitlements

To demonstrate entitlements, we'll need to have access to two users:
- User A has admin rights and can see the property value by default. We will need this user to grant policies.
- User B is originally not entitled to see the property value.

The process is as follows:
- as User B, try to retrieve the transactions. This should return the transactions without including the property.
- as User A, grant User B access by applying a policy to their role
- as User B, try retrieving the transactions again. This should return the same transactions including the property.

### 3.1 Create a role

Here, we create a new role with two policies applied by default:
- `allow-standard-lusid-features-access`, which gives the user access to standard LUSID features. Without this, the user wouldn't be able to retrieve transactions at all.
- `allow-scope-access-PropertyValueEntitlements`, which gives the user access to the scope in which our TransactionPortfolio sits. Without this, the user would not be able to see anything in the `PropertyValueEntitlements` scope.

Notably, neither of these policies grants explicit access to the property we created above, so users assigned this role will not be entitled to see it.

In [9]:
# WhenSpec objects specify the "lifetime" of a modification:
# when it is activated and when it is deactivated.
when_spec = access_models.WhenSpec(
    activate=datetime.now(tz=pytz.utc) - timedelta(days=2),
    deactivate=datetime(9999, 12, 31, tzinfo=pytz.utc),
)

In [10]:
role_code = "PropertyValueEntitlementsRole"
allow_features_policy_code = "allow-standard-lusid-features-access"
allow_scope_policy_code = f"allow-scope-access-{scope}"

role_creation_request = access_models.RoleCreationRequest(
    code=role_code,
    description=role_code,
    resource=access_models.RoleResourceRequest(
        policy_id_role_resource=access_models.PolicyIdRoleResource(
            policies=[
                access_models.PolicyId(scope=scope, code=allow_features_policy_code),
                access_models.PolicyId(
                    scope=scope,
                    code=allow_scope_policy_code,
                ),
            ]
        )
    ),
    when=when_spec,
)


try:
    response = roles_api.create_role(role_creation_request)
except finbourne_access.ApiException as e:
    detail = json.loads(e.body)
    if detail["code"] != 613:  # RoleWithCodeAlreadyExists
        raise e

### 3.2 Create a policy to allow access to the new property

In [11]:
entities = ["Transaction"]
property_value_policy_code = f"allow-transaction-property-access-in-{scope}"

# Get access path where to apply the policy.
scope_selector_definition = access_models.IdSelectorDefinition(
    identifier={"scope": scope, "code": property_code, "domain": property_domain},
    actions=[
        access_models.ActionId(scope="default", activity="Any", entity="PropertyValue")
    ],
    name=f"{scope} scope",
    description=f"{scope} scope",
)

allow_scope_path = access_models.SelectorDefinition(
    id_selector_definition=scope_selector_definition
)
allow_scope_policy_request = access_models.PolicyCreationRequest(
    code=property_value_policy_code,
    description=f"Allows access to {property_domain}/{scope}/{property_code}",
    applications=["LUSID"],
    grant="Allow",
    selectors=[allow_scope_path],
    when=when_spec,
)

try:
    # Create the policy.
    policies_api.create_policy(allow_scope_policy_request)
except finbourne_access.ApiException as e:
    detail = json.loads(e.body)
    if detail["code"] != 613:  # PolicyWithCodeAlreadyExists
        raise e

### 3.3 Retrieve transactions with property

First, let's try to retrieve the transactions with User B's current entitlements. To do so, run the following statement as User B.

In [12]:
transaction_portfolios_api.get_transactions(scope=scope, code=portfolio_code)

{'href': 'https://inwaves.lusid.com/api/api/transactionportfolios/PropertyValueEntitlements/PropertyEntitlementsPortfolioCode/transactions?effectiveAt=9999-12-31T23%3A59%3A59.9999999%2B00%3A00&asAt=2021-04-09T08%3A19%3A35.0187140%2B00%3A00',
 'links': [{'description': None,
            'href': 'https://inwaves.lusid.com/api/api/portfolios/PropertyValueEntitlements/PropertyEntitlementsPortfolioCode?effectiveAt=2021-04-08T13%3A57%3A58.1853940%2B00%3A00&asAt=2021-04-08T13%3A57%3A58.3342480%2B00%3A00',
            'method': 'GET',
            'relation': 'Root'},
           {'description': None,
            'href': 'https://inwaves.lusid.com/api/api/schemas/entities/Transaction',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': None,
            'href': 'https://inwaves.lusid.com/api/api/schemas/properties?propertyKeys=Transaction%2Fdefault%2FSourcePortfolioId%2CTransaction%2FPropertyValueEntitlements%2FPropertyValueEntitlement%2CTransaction%

To entitle User B, apply the property value policy we created above to their role by running the below statement as User A.

In [13]:
role_update_request = access_models.RoleUpdateRequest(
    description="Add new property value policy",
    resource=access_models.RoleResourceRequest(
        policy_id_role_resource=access_models.PolicyIdRoleResource(
            policies=[
                access_models.PolicyId(scope=scope, code=allow_features_policy_code),
                access_models.PolicyId(scope=scope, code=allow_scope_policy_code),
                access_models.PolicyId(scope=scope, code=property_value_policy_code),
            ]
        )
    ),
    when=when_spec,
)

roles_api.update_role(code=role_code, scope="default", role_update_request=role_update_request)

{'description': 'Add new property value policy',
 'id': {'code': 'PropertyValueEntitlementsRole', 'scope': 'default'},
 'limit': None,
 'links': [{'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://inwaves.lusid.com/app/insights/logs/0HM7QU0Q143UI:0000000A',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'permission': 'Read',
 'resource': {'non_transitive_supervisor_role_resource': None,
              'policy_id_role_resource': {'policies': [{'code': 'allow-standard-lusid-features-access',
                                                        'scope': 'PropertyValueEntitlements'},
                                                       {'code': 'allow-scope-access-PropertyValueEntitlements',
                                                        'scope': 'PropertyValueEntitlements'},
                                                       {'code': 'allow-trans

Finally, let's try retrieving the transactions as User B. These now contain the `Transaction/PropertyValueEntitlements/PropertyValueEntitlement` property.

In [14]:
transaction_portfolios_api.get_transactions(scope=scope, code=portfolio_code)

{'href': 'https://inwaves.lusid.com/api/api/transactionportfolios/PropertyValueEntitlements/PropertyEntitlementsPortfolioCode/transactions?effectiveAt=9999-12-31T23%3A59%3A59.9999999%2B00%3A00&asAt=2021-04-09T08%3A19%3A35.3742090%2B00%3A00',
 'links': [{'description': None,
            'href': 'https://inwaves.lusid.com/api/api/portfolios/PropertyValueEntitlements/PropertyEntitlementsPortfolioCode?effectiveAt=2021-04-08T13%3A57%3A58.1853940%2B00%3A00&asAt=2021-04-08T13%3A57%3A58.3342480%2B00%3A00',
            'method': 'GET',
            'relation': 'Root'},
           {'description': None,
            'href': 'https://inwaves.lusid.com/api/api/schemas/entities/Transaction',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': None,
            'href': 'https://inwaves.lusid.com/api/api/schemas/properties?propertyKeys=Transaction%2FPropertyValueEntitlements%2FPropertyValueEntitlement%2CTransaction%2Fdefault%2FSourcePortfolioId%2CTransaction%